In [1]:
# pip install torch torchvision torchsummary
# pip install scikit-learn

# Import Libraries

In [2]:
import torchvision
import torch.utils.data as data
import torchvision.transforms as transforms
import torch
import numpy as np
import copy
from torchsummary import summary
from torch.optim import lr_scheduler
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import time
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

SEED = 1234

#random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Preprocessing

In [3]:
# Transformations to be added to the train dataset
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])


# Transformations to be added to the test dataset
# Test transformations 
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

In [ ]:
###################### Train and valid Loader ######################
trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=train_transform)

# Validation Data ratio from Train data
VALID_RATIO = 0.8
# Number of train samples
n_train_examples = int(len(trainset) * VALID_RATIO)
# Number of test samples
n_valid_examples = len(trainset) - n_train_examples

# Dividing train and validation based on number of samples
train_data, valid_data = data.random_split(trainset, 
                                           [n_train_examples, n_valid_examples])

# Train Data Loader
trainloader = torch.utils.data.DataLoader(
    train_data, batch_size=32, shuffle=True, num_workers=2)

# Applying test transform on validation data after a deepcopy
valid_data = copy.deepcopy(valid_data)
valid_data.dataset.transform = test_transform

# Validation Data Loader
validloader = torch.utils.data.DataLoader(
    valid_data, batch_size=1024, shuffle=False, num_workers=2)

###################### Test Loader ######################
testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=test_transform)

testloader = torch.utils.data.DataLoader(
    testset, batch_size=500, shuffle=False, num_workers=2)


###################### Datalaoders ######################
dataloaders = {'train': trainloader,  'valid': validloader, 'test': testloader}
dataset_sizes = {'train': len(trainloader.dataset), 'valid': len(validloader.dataset), 'test': len(testloader.dataset)}

In [5]:
# Class names in the CIFAR10 dataset
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

# Model Definition

In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        # Conv followed by BN
        # Since image size is (32, 32), kernel size is kept small with limited padding
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=True)
        self.bn1 = nn.BatchNorm2d(planes)
         # second batch of conv layers followed by BN
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=True)
        self.bn2 = nn.BatchNorm2d(planes)

        # Sequential layer to represent the skip connection
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, padding=0, stride=stride, bias=True),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        # Tried out - ReLU and Leaky Relu where results were pretty similar
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        # Here is the skip connection
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 32
        
        # First conv layer followed by BN
        # Size has been kept at 32, to facillitate higher number of layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3,
                               stride=1, padding=1, bias=True)
        self.bn1 = nn.BatchNorm2d(32)
        
        # Make 4 batches of layers with different input size
        # Have kept the block sizes power of 2s
        # Added a few dropout layers for the 
        self.layer1 = self._make_layer(block, 32, num_blocks[0], stride=1)
        self.drop_out_1 = nn.Dropout(p=0.5)
        self.layer2 = self._make_layer(block, 64, num_blocks[1], stride=2)
        self.drop_out_2 = nn.Dropout(p=0.5)
        self.layer3 = self._make_layer(block, 128, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 256, num_blocks[3], stride=2)
        # Sequential layer to map final block size to number of classes
        self.linear = nn.Linear(256*block.expansion, num_classes)

    # Building basic blocks in a modular fashion
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        # We experimented with the order of dropout within these layers
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.drop_out_1(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = self.drop_out_2(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [7]:
# Config for different sized blocks
config = [4, 4, 4, 3]

# Model Creation
model = ResNet(BasicBlock, config)

print("Number of trainable parameters:", sum(p.numel() for p in model.parameters()))

Number of trainable parameters: 4758026


In [ ]:
# Kaiming initialization for Conv Layer and general initialization for BN and Linear
def initialize_parameters(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight.data, mode='fan_out', nonlinearity = 'relu')
        nn.init.constant(m.bias.data, 0)
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant(m.weight.data, 1)
        nn.init.constant(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        nn.init.normal(m.weight.data, std=1e-3)
        nn.init.constant(m.bias.data, 0)

# Applying Initializations
model.apply(initialize_parameters)

# Parameterization

In [9]:
# Detecting which device to train the model on
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load model onto device
model = model.to(device)

In [10]:
# Loss function
# For multiclass image classification, crossentropy worked great compared to multimargin loss
criterion = nn.CrossEntropyLoss(reduction='sum')

# Epochs
EPOCH = 100

# Learning rate
lr = 0.01

# weight decay
weightDecay = 0.0001

# Optimizer
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weightDecay)

# Schedulers
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor = 0.5, patience=5, cooldown=2)

# Training

In [11]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    
    start = time.time()
    # Storing loss and accuracy to be used later to plot
    train_losses, valid_losses = [], []
    train_acc, valid_acc = [], []

    # Copying the initial network weights
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        nn.utils.clip_grad_norm_(model.parameters(), 5)
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step(running_loss)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
            
            # Storing every train and validation epoch accuracy and loss
            if phase == 'train':
                train_losses.append(epoch_loss)
                train_acc.append(epoch_acc)
            elif phase == 'valid':
                valid_losses.append(epoch_loss)
                valid_acc.append(epoch_acc)            
            
        
    time_elapsed = time.time() - start
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')


    # load best model weightsssss
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
# Here we are training the model 3 times sequentially to manually simulate a restart
first_run_train_losses, first_run_valid_losses, first_run_train_acc, first_run_valid_acc, model = train_model(model, 
                                                                                                     criterion, 
                                                                                                     optimizer, 
                                                                                                     scheduler, 
                                                                                                     EPOCH)

PATH = "training_3_1_f.pt"
torch.save(model.state_dict(), PATH)

# 1st Restart
second_run_train_losses, second_run_valid_losses, second_run_train_acc, second_run_valid_acc, model = train_model(model, 
                                                                                                     criterion, 
                                                                                                     optimizer, 
                                                                                                     scheduler, 
                                                                                                     EPOCH)

PATH = "training_3_2_f.pt"
torch.save(model.state_dict(), PATH)

# 2nd Restart
lr = optimizer.param_groups[0]['lr']
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weightDecay)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor = 0.5, patience=5, cooldown=2)
third_run_train_losses, third_run_valid_losses, third_run_train_acc, third_run_valid_acc, model = train_model(model, 
                                                                                                     criterion, 
                                                                                                     optimizer, 
                                                                                                     scheduler, 
                                                                                                     EPOCH)

PATH = "training_3_3_f.pt"
torch.save(model.state_dict(), PATH)

In [ ]:
# Appending all the train losses in a single list
train_losses = first_run_train_losses + second_run_train_losses + third_run_train_losses
# Appending all the validation losses in a single list
valid_losses = first_run_valid_losses + second_run_valid_losses + third_run_valid_losses

# Appending all the train accuracy in a single list
train_acc = first_run_train_acc + second_run_train_acc + third_run_train_acc
# Appending all the validation accuracy in a single list
valid_acc = first_run_valid_acc + second_run_valid_acc + third_run_valid_acc

# COnverting from tensor to list
train_acc = [i.cpu().tolist() for i in train_acc]
valid_acc = [i.cpu().tolist() for i in valid_acc]

# Creating a pandas dataframe for better processing
results_df = pd.DataFrame({'train_loss':train_losses, 'valid_loss':valid_losses, 
              'train_acc':train_acc, 'valid_acc':valid_acc})

# Plotting the Loss for train and validation
plt.figure(figsize=(10, 10))
df[['train_loss', 'valid_loss']].plot()
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.title("Train VS Validation Loss")
plt.savefig("Loss_plot.jpg", dpi = 160)

# Plotting the accuracy for train and validation
plt.figure(figsize=(10, 10))
df[['train_acc', 'valid_acc']].plot()
plt.xlabel("Number of Epochs")
plt.ylabel("Accuracy")
plt.title("Train VS Validation Accuracy")
plt.savefig("Acc_plot.jpg", dpi = 160)

# Inference

In [ ]:
# Function to calculate performance metrics of results
def performance_metrics(labels, preds, classes):
    print("Overall metrics")
    print("Accuracy:", accuracy_score(labels, preds))
    print("Recall:", recall_score(labels, preds, average='weighted'))
    print("Precision:", precision_score(labels, preds, average='weighted'))
    print("F1:", f1_score(labels, preds, average='weighted'))
    
    print("\nIndividual Metrics\n")
    recall_scores = recall_score(labels, preds, average=None)
    precision_scores = precision_score(labels, preds, average=None)
    f1_scores = f1_score(labels, preds, average=None)
    return pd.DataFrame({'Class': classes, 'Recall': recall_scores, 'Precision':precision_scores,
                 'F1':f1_scores})

In [14]:
def test_model(model):
    
    # Save last state of training before changin to eval mode
    # This is done to resume training after eval, in case
    was_training = model.training
    # Eval mode of the model. So, it doesn't learn the  discrepencies
    model.eval()
    
    # Empty list to hold values
    all_labels = []
    all_preds = []

    
    # Generates the outputs and stores it in the list
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['test']):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Gernerating predictions
            outputs = model(inputs)
            # Acquiring predictions
            _, preds = torch.max(outputs, 1)
            all_labels.append(labels)
            all_preds.append(preds)
    
    # Model returned to training state
    model.train(mode=was_training)
    return all_labels, all_preds
    
labels, preds = test_model(model)

In [15]:
# Converting to better formats for calculating performance metrics
labels = [j  for i in labels for j in i.cpu().tolist()[:]]
preds = [j  for i in preds for j in i.cpu().tolist()[:]]

In [16]:
# Results
performance_metrics(labels, preds, classes)

Overall metrics
Accuracy: 0.9365
Recall: 0.9365
Precision: 0.936397965853499
F1: 0.9364173162851157

Individual Metrics



,Class,Recall,Precision,F1
0,plane,0.943,0.933663,0.938308
1,car,0.973,0.957677,0.965278
2,bird,0.911,0.918347,0.914659
3,cat,0.857,0.878074,0.867409
4,deer,0.946,0.933860,0.939891
5,dog,0.909,0.900000,0.904478
6,frog,0.954,0.956871,0.955433
7,horse,0.955,0.964646,0.959799
8,ship,0.954,0.955912,0.954955
9,truck,0.963,0.964930,0.963964


We can see that the overall accuracy is 93.65%. We also see that the cat and dog classes didn't perform well based on their Recall, Precision and F1 Score. 